In [35]:
import os
import subprocess
import pandas as pd
from Bio import SeqIO

This one needs some work. Could be improved to automaticall rename. Should be pipelined into handling ncbi genome downloads?

In [38]:
blastdir = '/home/ecutts/Bacteroidetes/Chlorobi/blastdb/' # dir containing blast database (recocomended: absolute path)
query = '/home/ecutts/Bacteroidetes/Chlorobi/queries/bfragilis-30rprots.faa' # path to query (reccomended: absolute path)
odir = '/home/ecutts/Bacteroidetes/Chlorobi/results/' # directory for blast output
fastadir = '/home/ecutts/Bacteroidetes/Chlorobi/rprots/' # directory for fasta files with output

# path to blast command
blast = '/cm/shared/engaging/ncbi-blast/2.6.0+/bin/'
blastp = blast + 'blastp' # change if you want to use a different blast (i.e. tblastn)

# extension of blast database sequence files (e.x. .fasta, .faa)
ext = '.faa' 

# short name of query
q_shortname = 'rprots'

In [20]:
os.chdir(blastdir)
for db in os.listdir():
    if db.endswith(ext): # only check the blast db files
        db_id = db.split('_protein')[0] # change this code to pull out the id of your seqs
        p = subprocess.run([blastp, '-query', query,\
                '-db', db, '-out', odir + q_shortname + '_vs_' + db_id + '.out',\
                '-outfmt', '6'])
        print(p)

CompletedProcess(args=['/cm/shared/engaging/ncbi-blast/2.6.0+/bin/blastp', '-query', '/home/ecutts/Bacteroidetes/Chlorobi/queries/bfragilis-30rprots.faa', '-db', 'GCF_000012485.1_ASM1248v1_protein.faa', '-out', '/home/ecutts/Bacteroidetes/Chlorobi/results/rprots_vs_GCF_000012485.1_ASM1248v1.out', '-outfmt', '6'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/ncbi-blast/2.6.0+/bin/blastp', '-query', '/home/ecutts/Bacteroidetes/Chlorobi/queries/bfragilis-30rprots.faa', '-db', 'GCF_000012585.1_ASM1258v1_protein.faa', '-out', '/home/ecutts/Bacteroidetes/Chlorobi/results/rprots_vs_GCF_000012585.1_ASM1258v1.out', '-outfmt', '6'], returncode=0)
CompletedProcess(args=['/cm/shared/engaging/ncbi-blast/2.6.0+/bin/blastp', '-query', '/home/ecutts/Bacteroidetes/Chlorobi/queries/bfragilis-30rprots.faa', '-db', 'GCF_000015125.1_ASM1512v1_protein.faa', '-out', '/home/ecutts/Bacteroidetes/Chlorobi/results/rprots_vs_GCF_000015125.1_ASM1512v1.out', '-outfmt', '6'], returncode=0)
CompletedProc

In [64]:
# parse blast results into dataframes and ouptput sequences results to fasta files
# files have same extension as input dbs

os.chdir(odir)

dfs = {}
header = ['query acc.ver', 'subject acc.ver', '% identity', 'alignment length', 'mismatches',\
          'gap opens', 'q. start', 'q. end', 's. start', 's. end', 'evalue', 'bit score']

def find_fasta(name):
    for f in os.listdir(blastdir):
        if f.startswith(name):
            return f
    
    
for outfile in os.listdir(): # read outputs into dataframes
    if outfile.endswith('.out'):
        name = outfile.split('_vs_')[1].split('.out')[0]
        dfs[name] = pd.read_table(outfile, names=header)
        dfs[name] = dfs[name][((dfs[name]['% identity'] >= 80)|(dfs[name]['evalue'] < 0.05))] # filter for e value or identity
        print(name + ': ' + str(len(dfs[name])) + ' hits')
        
        # write sequences to output fasta
        fasta = fastadir + name + ext
        records = []
        with open(fasta, 'w') as ofasta, open(blastdir + find_fasta(name), 'r') as ifasta:
            for record in SeqIO.parse(ifasta, 'fasta'):
                for index, row in dfs[name].iterrows(): 
                    if row['subject acc.ver'] == record.id:
                        records.append(record)
            SeqIO.write(records, ofasta, 'fasta')
        

GCF_000012485.1_ASM1248v1: 30 hits
GCF_000012585.1_ASM1258v1: 32 hits
GCF_000015125.1_ASM1512v1: 31 hits
GCF_000016085.1_ASM1608v1: 34 hits
GCF_000020465.1_ASM2046v1: 31 hits
GCF_000020505.1_ASM2050v1: 30 hits
GCF_000020525.1_ASM2052v1: 33 hits
GCF_000020545.1_ASM2054v1: 32 hits
GCF_000020625.1_ASM2062v1: 31 hits
GCF_000020645.1_ASM2064v1: 31 hits
GCF_001687065.1_ASM168706v1: 33 hits
GCF_001747405.1_ASM174740v1: 30 hits
GCF_002113825.1_ASM211382v1: 32 hits
GCF_002240205.1_ASM224020v1: 32 hits
GCF_000006985.1_ASM698v1: 30 hits


In [90]:
# optional: check for riboprots and rename according to taxid
# my normal rprot format is rprots_TAXID_ASSEMBLY.fasta

os.chdir(fastadir)
rprots = pd.read_csv('/home/ecutts/metadata/rprots.txt', header = None)[0].tolist()

# read NCBI assembly reports
def read_report(report):
    assembly_report = open(report, 'r')

    while True:
        line = assembly_report.readline()
        if not line:
            break

        if 'Organism name' in line: 
            name = line.split('Organism name:  ')[1][:-1]
            name = name.split(' (')[0].replace(' ', '_')

        if 'GenBank assembly accession' in line:
            genbank_acc = line.split(' GenBank assembly accession: ')[1][:-1]

        if 'Taxid' in line:
            taxid = line.split('Taxid:          ')[1][:-1]
    
    return {'name': name, 'taxid': taxid, 'genbank': genbank_acc}

def find_report(d):
    for f in os.listdir(d):
        if f.endswith('assembly_report.txt'):
            return f

def get_taxid(searchdir):
    return read_report(searchdir + '/' + find_report(searchdir))['taxid']
    

for f in os.listdir():
    print(f)
    assembly =f.split('_')[0] + '_' + f.split('_')[1]
    searchdir = '/home/ecutts/Bacteroidetes/Chlorobi/refseq/bacteria/' + assembly
    taxid = get_taxid(searchdir)
    name = 'rprots_' + taxid + '_' + assembly + '.fasta'
    records = []
    counts = {}
    with open(f, 'r') as fasta, open(name, 'w') as outfasta:
        counts = dict.fromkeys(rprots)
        for prot in counts:
            counts[prot] = 0
        for record in SeqIO.parse(fasta, 'fasta'):
            if 'ribosomal protein' in record.description:
                prot = record.description.split('protein ')[1].split(' [')[0]
                counts[prot] += 1
                records.append(record)
        SeqIO.write(records, outfasta, 'fasta')
        
        for key in counts.keys():
            if counts[key] != 1:
                print(key + ': ' + str(counts[key]))

GCF_000012485.1_ASM1248v1.faa
L2: 0
GCF_000012585.1_ASM1258v1.faa
GCF_000015125.1_ASM1512v1.faa
GCF_000016085.1_ASM1608v1.faa
GCF_000020465.1_ASM2046v1.faa
L29: 0
GCF_000020505.1_ASM2050v1.faa
L22: 0
GCF_000020525.1_ASM2052v1.faa
GCF_000020545.1_ASM2054v1.faa
GCF_000020625.1_ASM2062v1.faa
GCF_000020645.1_ASM2064v1.faa
L29: 0
GCF_001687065.1_ASM168706v1.faa
GCF_001747405.1_ASM174740v1.faa
GCF_002113825.1_ASM211382v1.faa
S13: 0
GCF_002240205.1_ASM224020v1.faa
GCF_000006985.1_ASM698v1.faa
